In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/plbart_base_embeddings/processed_embeddings/plbart_embed_line_by_line_avg.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,0.765308,-0.861020,0.346896,-1.750568,2.594928,-2.730383,-0.880609,-0.756988,-0.500890,-0.597879,...,-0.097720,0.161241,-2.178341,-3.397756,-2.166048,2.057266,1.184136,1,8077,critical
1,1.805521,-0.084213,-1.366126,-1.700137,2.654870,-1.521022,-1.205181,-0.380706,-1.626614,-1.930437,...,-1.490124,0.111386,-1.366581,-0.949441,-3.066421,1.671481,-1.336844,1,5553,critical
2,3.370630,-1.042931,-1.150805,-1.078694,2.299146,-1.086777,-1.214361,0.185659,-1.320936,-2.077391,...,1.227185,1.222121,-1.098609,-3.634563,-2.632948,0.881600,1.248681,1,9341,critical
3,1.924327,-1.098289,0.269451,-1.364447,1.807756,-1.252493,-0.891735,-0.757447,-0.782254,-0.210271,...,-0.628672,0.635927,-1.499439,-3.881006,-1.574075,2.070461,0.875936,1,10419,critical
4,0.765308,-0.861020,0.346896,-1.750568,2.594928,-2.730383,-0.880609,-0.756988,-0.500890,-0.597879,...,-0.097720,0.161241,-2.178341,-3.397756,-2.166048,2.057266,1.184136,1,12232,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8333
F1_macro: 0.7781
Accuracy: 0.8333
Precision_macro: 0.7905
Recall_macro: 0.7687

Bagging Metrics:
F1_micro: 0.8321
F1_macro: 0.7699
Accuracy: 0.8321
Precision_macro: 0.7941
Recall_macro: 0.7549

XGBoost Metrics:
F1_micro: 0.8251
F1_macro: 0.7720
Accuracy: 0.8251
Precision_macro: 0.7770
Recall_macro: 0.7679


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.8233
F1_macro: 0.7691
Precision_macro: 0.7750
Recall_macro: 0.7640
Accuracy: 0.8233
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       814
           1       0.68      0.64      0.66       295

    accuracy                           0.82      1109
   macro avg       0.78      0.76      0.77      1109
weighted avg       0.82      0.82      0.82      1109


Bagging Metrics:
F1_micro: 0.8188
F1_macro: 0.7596
Precision_macro: 0.7704
Recall_macro: 0.7512
Accuracy: 0.8188
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       814
           1       0.68      0.61      0.64       295

    accuracy                           0.82      1109
   macro avg       0.77      0.75      0.76      1109
weighted avg       0.81      0.82      0.82      1109


XGBoost Metrics:
F1_micro: 0.8341
F1_macro: 

In [6]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/plbart/result_plbart_avg.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/plbart/result_plbart_avg.pkl
